# Alternating Least Squares

## References

http://cs229.stanford.edu/proj2017/final-posters/5147271.pdf

## Intro 

ALS algorithm works by alternating between rows and columns to factorized the matrix.

Stochastic Gradient Descent 

- Flexibility
- Parallel
- Slower
- Hard to handle unobserved interaction (sparsity)

Alternating Least Square 

- basically ordinary least square method only
- Parallel
- Faster
- Easy to handle sparsity


## Algorithm

1. Initiate row factor U, column factor V
2. Repeat until convergence
    1. for i = 1 to n do   (iterating over rows)
        
        $u_i = (\sum_{r_{i,j} \in r_{i*}} {v_j v_j^T + \lambda I_k })^{-1} \sum_{r_{i,j} \in r_{i*}}{r_{ij} v_j}$
        
       end for  [solving for row factors when column factors are features]
    
    2. for i = 1 to m do   (iterative over columns)
       
        $v_i = (\sum_{r_{i,j} \in r_{*j}} {u_i u_i^T + \lambda I_k })^{-1} \sum_{r_{i,j} \in r_{*j}}{r_{ij} u_i}$
       
       end for [solving for column factors when row factors are features]
       
       
Where\
    U = row factor matrix\
    V = column factor matrix\
    r = ratings
    
    
    
Similarity between als vs ols with l2 regularization 

$\theta = (X^T X + \lambda I)^{-1} X^T Y$

$v_i = (\sum_{r_{{i,j} \in r_{*j}}} {u_i u_i^T + \lambda I_k })^{-1} \sum_{r_{i,j} \in r_{*j}}{r_{ij} u_i}$

## Approach


```sql
    
              movies (n)                         k                  n
            +---------------------+        +------+   +----------------------+
            |                     |        |      | x |      movie           | k
            |                     |        | user |   |      factor          |      V
            |                     |        |factor|   +----------------------+
  users(m)  |                     |        |      |
            |     RATINGS         |    ~   |      |
            |                     |    ~   |      | m
            |                     |        |      |
            |                     |        |      |
            |                     |        |      |
            +---------------------+        +------+
                                                U
```

Iterative Algorithm 

* fix V, compute U
* fix U, compute V

## Compare with SVD


Matrix Factorization Method.

More here https://machinelearningexploration.readthedocs.io/en/latest/MathExploration/SingularValueDecomposition.html

```sql

                                        +----------------------+
                                      K |______________________| 
                                        |                      |  
  items                            K    +----------------------+        
+---------------------+        +-------+   
|                     |        |   |   |   
|u                    |        |   |   |   
|s                    |        |   |   |
|e                    |    ~   |   |   |
|r                    |    ~   |   |   | 
|s                    |        |   |   |
|                     |        |   |   |
|                     |        |   |   |
+---------------------+        +-------+
           X                      V       x       VT             


                 row factors (items embeddings)
               /
Factorization 
               \ 
                column factors (user embeddings)
                
                
```


In SVD the missing observations has to be fill as zeros.

$| A - U V^T |^2$

```sql
        item1  item2  item3   item4
       +------+------+------+------+
 user1 |  1   |  0   |  0   |  1   |
       +------+------+------+------+
 user2 |  0   |  1   |  0   |  0   |
       +------+------+------+------+
 user3 |  0   |  1   |  0   |  0   |
       +------+------+------+------+
 user4 |  0   |  0   |  1   |  0   |
       +------+------+------+------+
   
```


In ALS we use rows and columns alternatively as features, hence `no need to fill missing values`.

$\sum_{i,j \in obs} (A_{ij} - U_i V_j^T)^2$

```sql
        item1  item2  item3   item4
       +------+------+------+------+
 user1 |  1   |      |      |  1   |
       +------+------+------+------+
 user2 |      |  1   |      |      |
       +------+------+------+------+
 user3 |      |  1   |      |      |
       +------+------+------+------+
 user4 |      |      |  1   |      |
       +------+------+------+------+
   
```


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [3]:
ratings = pd.read_csv('/opt/datasetsRepo/RecommendationData/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
idx_to_userid_mapper = dict(enumerate(ratings.userId.unique()))
userid_to_idx_mapper = dict(zip(idx_to_userid_mapper.values(), idx_to_userid_mapper.keys()))

In [5]:
idx_to_movieid_mapper = dict(enumerate(ratings.movieId.unique()))
movieid_to_idx_mapper = dict(zip(idx_to_movieid_mapper.values(), idx_to_movieid_mapper.keys()))

In [7]:
len(idx_to_userid_mapper), len(userid_to_idx_mapper)

(610, 610)

In [8]:
len(idx_to_movieid_mapper), len(movieid_to_idx_mapper)

(9724, 9724)

In [10]:
ratings['user_idx'] = ratings['userId'].map(userid_to_idx_mapper).apply(np.int32)
ratings['movie_idx'] = ratings['movieId'].map(movieid_to_idx_mapper).apply(np.int32)
ratings.head(10)

,userId,movieId,rating,timestamp,user_idx,movie_idx
0,1,1,4.0,964982703,0,0
1,1,3,4.0,964981247,0,1
2,1,6,4.0,964982224,0,2
3,1,47,5.0,964983815,0,3
4,1,50,5.0,964982931,0,4
5,1,70,3.0,964982400,0,5
6,1,101,5.0,964980868,0,6
7,1,110,4.0,964982176,0,7
8,1,151,5.0,964984041,0,8
9,1,157,5.0,964984100,0,9


In [30]:
user_movie_matrix = ratings.pivot_table(values=['rating'] ,index=['user_idx'], 
                                        columns=['movie_idx'])

In [31]:
user_movie_matrix.head()

rating                                               ...            \
movie_idx   0    1    2    3    4    5    6    7    8    9     ... 9714 9715   
user_idx                                                       ...             
0            4.0  4.0  4.0  5.0  5.0  3.0  5.0  4.0  5.0  5.0  ...  NaN  NaN   
1            NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
2            NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
3            NaN  NaN  NaN  2.0  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
4            4.0  NaN  NaN  NaN  4.0  NaN  NaN  4.0  NaN  NaN  ...  NaN  NaN   

                                                   
movie_idx 9716 9717 9718 9719 9720 9721 9722 9723  
user_idx                                           
0          NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
1          NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
2          NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
3          NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
4          NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[5 rows x 9724 columns]

In [56]:
np.int32(user_movie_matrix > 1)

array([[1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 1, 1, 1]], dtype=int32)

In [70]:
u, s, vT = np.linalg.svd(user_movie_matrix.fillna(0).values, full_matrices=False)

In [114]:
r = 
a = u[:,:r] @ np.diag(s[:r]) @ vT[:r,:]

np.sqrt(np.square(a - user_movie_matrix.fillna(0).values).sum())

509.568283237293

# Weighted Alternating Least Squares (WALS)

\begin{align}
    \sum_{i,j \in obs} (A_{ij} - U_i V_j)^2 - w_k \times \sum_{i,j \notin obs} (0 - U_i V_j)^2
\end{align}